In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy import distance

In [2]:
### This data can be downloaded from https://www.paoilandgasreporting.state.pa.us/publicreports/Modules/Welcome/Agreement.aspx
### Agree to the statement, select "Production/Waste Reports" and then select "Waste Report"
### Select the desired months.  This data file was selected from all of 2018 and then reduced to 25% of the observations to save space

waste_report = pd.read_csv('Well Waste 2018 - Partial.csv', encoding='iso-8859-1', low_memory = False)

In [3]:
# Basic data and column reformatting
waste_report.rename(str.lower, axis='columns', inplace = True)
waste_report.rename(columns={'units':'waste_unit'}, inplace=True)
waste_report.waste_unit = waste_report.waste_unit.astype(str)
waste_report[['waste_quantity']] = waste_report[['waste_quantity']].astype(float)
waste_report.waste_facility_name = waste_report.waste_facility_name.fillna("")
waste_report.waste_type = waste_report.waste_type.str.split("(").str[0]

In [4]:
# Creates two new columns to separate the waste quantities into solid and liquid
waste_report['solid_waste'] =0
waste_report['liquid_waste']=0

# Moves values with "Tons" as a waste unit into 'solid_waste'
waste_report.loc[waste_report.waste_unit  == 'Tons', 'solid_waste'] = waste_report.waste_quantity[waste_report.waste_unit  == 'Tons']

#MOves values with "Bbl" as a waste unit into 'liquid_waste'
waste_report.loc[waste_report.waste_unit  == 'Bbl', 'liquid_waste'] = waste_report.waste_quantity[waste_report.waste_unit  == 'Bbl']

# Removes the original 'waste_quantity' column
waste_report = waste_report.drop(columns=['waste_quantity'])

In [5]:
# Fills in the well pad's geolocation as the facility geolocation when the well pad is the recipient of the waste produced

waste_report.loc[waste_report.disposal_method == 'REUSE (AT WELL PAD)', 'facility_latitude'
                ] = waste_report.well_pad_latitude[waste_report.disposal_method == 'REUSE (AT WELL PAD)']
waste_report.loc[waste_report.disposal_method == 'REUSE (AT WELL PAD)', 'facility_longitude'
                ] = waste_report.well_pad_longitude[waste_report.disposal_method == 'REUSE (AT WELL PAD)']

In [6]:
# Fills in a somewhat arbitrary location for waste transported out of Pennsylvania
# The Ohio and West Virginia locations are in the general region of significant oil and gas production

waste_report.loc[waste_report.waste_facility_name == 'REUSE AT OHIO WELL PAD', 'facility_latitude'] = 40
waste_report.loc[waste_report.waste_facility_name == 'REUSE AT OHIO WELL PAD', 'facility_longitude'] = -81

waste_report.loc[waste_report.waste_facility_name == 'REUSE AT TEXAS WELL PAD', 'facility_latitude'] = 32
waste_report.loc[waste_report.waste_facility_name == 'REUSE AT TEXAS WELL PAD', 'facility_longitude'] = -95

waste_report.loc[waste_report.waste_facility_name == 'REUSE AT VIRGINIA WELL PAD', 'facility_latitude'] = 37
waste_report.loc[waste_report.waste_facility_name == 'REUSE AT VIRGINIA WELL PAD', 'facility_longitude'] = -81

waste_report.loc[waste_report.waste_facility_name == 'REUSE AT WEST VIRGINIA WELL PAD', 'facility_latitude'] = 39.6
waste_report.loc[waste_report.waste_facility_name == 'REUSE AT WEST VIRGINIA WELL PAD', 'facility_longitude'] = -81

waste_report.loc[waste_report.waste_facility_name == 'REUSE AT WYOMING WELL PAD', 'facility_latitude'] = 42
waste_report.loc[waste_report.waste_facility_name == 'REUSE AT WYOMING WELL PAD', 'facility_longitude'] = -105

In [7]:
# Replace the period ID with standard year-month format

waste_report.loc[waste_report.period_id == '18JANW', 'period_id' ] = '2018-01'

waste_report.loc[waste_report.period_id == '18FEBW', 'period_id' ] = '2018-02'

waste_report.loc[waste_report.period_id == '18MARW', 'period_id' ] = '2018-03'

waste_report.loc[waste_report.period_id == '18APRW', 'period_id' ] = '2018-04'

waste_report.loc[waste_report.period_id == '18MAYW', 'period_id' ] = '2018-05'

waste_report.loc[waste_report.period_id == '18JUNW', 'period_id' ] = '2018-06'

waste_report.loc[waste_report.period_id == '18JULW', 'period_id' ] = '2018-07'

waste_report.loc[waste_report.period_id == '18AUGW', 'period_id' ] = '2018-08'

waste_report.loc[waste_report.period_id == '18SEPW', 'period_id' ] = '2018-09'

waste_report.loc[waste_report.period_id == '18OCTW', 'period_id' ] = '2018-10'

waste_report.loc[waste_report.period_id == '18NOVW', 'period_id' ] = '2018-11'

waste_report.loc[waste_report.period_id == '18DECW', 'period_id' ] = '2018-12'

In [8]:
waste_report.head()

,well_permit_num,period_id,well_status,waste_type,waste_unit,disposal_method,owner,ogo,well_county,well_municipality,...,waste_facility_permit_num,waste_facility_name,facility_latitude,facility_longitude,well_pad,well_pad_latitude,well_pad_longitude,waste_comment,solid_waste,liquid_waste
0,051-23718,2018-12,Active,Produced Fluid,Bbl,INJECTION DISPOSAL WELL,DIVERSIFIED OIL & GAS LLC,OGO-68792,Fayette,German,...,2D0830792,H MCMULLAN 35A,38.83245,-80.00883,NaN,39.86377,-79.83009,NaN,0.0,90.0
1,051-23968,2018-12,Active,Produced Fluid,Bbl,INJECTION DISPOSAL WELL,DIVERSIFIED OIL & GAS LLC,OGO-68792,Fayette,Redstone,...,2D0830792,H MCMULLAN 35A,38.83245,-80.00883,NaN,39.93087,-79.87295,NaN,0.0,110.0
2,051-23977,2018-12,Active,Produced Fluid,Bbl,INJECTION DISPOSAL WELL,DIVERSIFIED OIL & GAS LLC,OGO-68792,Fayette,Washington,...,2D0830792,H MCMULLAN 35A,38.83245,-80.00883,NaN,40.12031,-79.80221,NaN,0.0,50.0
3,051-23983,2018-12,Active,Produced Fluid,Bbl,INJECTION DISPOSAL WELL,DIVERSIFIED OIL & GAS LLC,OGO-68792,Fayette,Washington,...,2D0830792,H MCMULLAN 35A,38.83245,-80.00883,NaN,40.12352,-79.80525,NaN,0.0,20.0
4,051-23984,2018-12,Active,Produced Fluid,Bbl,INJECTION DISPOSAL WELL,DIVERSIFIED OIL & GAS LLC,OGO-68792,Fayette,Washington,...,2D0830792,H MCMULLAN 35A,38.83245,-80.00883,NaN,40.09589,-79.80669,NaN,0.0,110.0
